# **DESCRIPTA.AI** -> Extension Tables Main: (main.py)

This script defines the Flask backend that receives ZIP files from a Google Apps Script client, processes them using a document analysis pipeline, and returns a ZIP archive with structured outputs (DOCX, JSON, XLSX, modified .tex). It serves as the bridge between the frontend (Google Sheets + Apps Script) and the table-processing logic (tab.py).

## Key Functionalities

---

@app.route('/run_pipeline', methods=['POST'])

This is the main endpoint exposed by the Flask app. It performs the following tasks:



1. ZIP Ingestion:

Accepts a file either as multipart/form-data (file field) or raw binary stream.

Validates the presence of the ZIP content.

2. Temporary Processing:

Saves the received ZIP file into a temporary directory (tempfile.TemporaryDirectory()).

Executes the pipeline defined in tab.py, using the helper function process_zip_and_generate_files(...).

3. Response:

Packages all generated outputs from the pipeline into a single ZIP archive.

Sends this archive back to the caller (usually the Apps Script client) using send_file.

4. Error Handling:

Logs full stack traces for any exceptions.

Returns a JSON-formatted error with the last few lines of the traceback.

## Auxiliary Details

---

In [ ]:
logging.basicConfig(...)

Sets up a basic logging format and verbosity level (INFO) for debugging and status output.

* NLTK Initialization:

Ensures required NLTK tokenizers (such as punkt) are available at runtime. Downloads missing resources automatically.



* Global Server Execution:



In [ ]:
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080)

Launches the Flask application on all interfaces (0.0.0.0) at port 8080, suitable for exposure via tools like ngrok.

## Dependencies
---

flask, nltk, zipfile, tempfile, traceback, glob

Custom module tab.py with the function process_zip_and_generate_files(...)

In [ ]:

import os
import tempfile
import traceback
import glob
import zipfile
import logging
import nltk

from flask import Flask, request, send_file, jsonify, abort
from Tab import process_zip_and_generate_files

# ─── Configuración logging y NLTK ──────────────────────────
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')
for resource in ['punkt', 'punkt_tab']:
    try:
        nltk.data.find(f'tokenizers/{resource}')
    except LookupError:
        nltk.download(resource)

app = Flask(__name__)

@app.route('/run_pipeline', methods=['POST'])
def run_pipeline():
    try:
        # 1) Intentar leer ZIP enviado como multipart/form-data
        if 'file' in request.files:
            zip_file_storage = request.files['file']
            zip_bytes = zip_file_storage.read()
        else:
            # 2) Fallback a raw body
            zip_bytes = request.get_data()

        if not zip_bytes:
            abort(400, description="No se recibió ningún ZIP")

        # 3) Crear carpeta temporal
        with tempfile.TemporaryDirectory() as tmp:
            in_path = os.path.join(tmp, 'input.zip')
            with open(in_path, 'wb') as f:
                f.write(zip_bytes)

            # 4) Ejecutar pipeline dentro de tmp
            cwd = os.getcwd()
            os.chdir(tmp)
            process_zip_and_generate_files(
                in_path,
                os.path.join(tmp, 'graphical.docx'),
                os.path.join(tmp, 'nongraphical.docx')
            )
            os.chdir(cwd)

            # 5) Empaquetar todos los resultados en un ZIP
            result_zip = os.path.join(tmp, 'resultados.zip')
            with zipfile.ZipFile(result_zip, 'w', zipfile.ZIP_DEFLATED) as zf:
                for ext in ('*.docx', '*.json', '*.xlsx'):
                    for path in glob.glob(os.path.join(tmp, ext)):
                        zf.write(path, arcname=os.path.basename(path))

            # 6) Devolver el ZIP final
            return send_file(
                result_zip,
                mimetype='application/zip',
                as_attachment=True,
                download_name='resultados.zip'
            )

    except Exception:
        traceback.print_exc()
        return jsonify({
            'error': 'Pipeline error',
            'trace': traceback.format_exc().splitlines()[-5:]
        }), 500

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080)
